In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/multi-lingual-sentiment-analysis/sample_submission.csv
/kaggle/input/multi-lingual-sentiment-analysis/train.csv
/kaggle/input/multi-lingual-sentiment-analysis/test.csv
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model.safetensors.index.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00003-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/config.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/LICENSE
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00001-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/README.md
/kaggle/input/llama-3.1/transformers/8b-instruct/2/USE_POLICY.md
/kaggle/input/llama-3.1/transformers/8b-instruct/2/tokenizer.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/tokenizer_config.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00004-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/special_tokens_map.json
/kaggle/input/llama-3.1/

In [2]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install unsloth

In [3]:
import pandas as pd
import numpy as np
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from sklearn.model_selection import train_test_split


In [4]:
from unsloth import FastLanguageModel
import torch


max_seq_length = 2048 * 2 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.



model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/kaggle/input/llama-3.1/transformers/8b-instruct/2", 
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.12: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla P100-PCIE-16GB. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 6.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/kaggle/input/llama-3.1/transformers/8b-instruct/2 does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


In [5]:
from trl import SFTTrainer

In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # 4x longer contexts auto supported!
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.2.12 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [7]:
from datasets import load_dataset

# Load CSV files correctly
dataset_train = load_dataset('csv', data_files="/kaggle/input/multi-lingual-sentiment-analysis/train.csv",split='train')
dataset_test = load_dataset('csv', data_files="/kaggle/input/multi-lingual-sentiment-analysis/test.csv")

# Ensure formatting_prompts_func is defined before calling map
# dataset_train=dataset_train
# dataset_train[0]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
map_dict = {
    "as": "Assamese", 
    "bd": "Bodo", 
    "bn": "Bengali", 
    "gu": "Gujarati",
    "hi": "Hindi", 
    "kn": "Kannada", 
    "ml": "Malayalam", 
    "mr": "Marathi",
    "or": "Odia", 
    "pa": "Punjabi", 
    "ta": "Tamil", 
    "te": "Telugu", 
    "ur": "Urdu"
}

In [9]:
def generate_prompt(language_data, num_examples=2):
    """
    Generates a prompt with few-shot examples drawn from the training data.

    Args:
        language_data (dict): A dictionary containing the 'sentence' and 'language' for the input.
        num_examples (int): The number of examples to include in the prompt for each language.

    Returns:
        str: The generated prompt.
    """
    full_language = map_dict[language_data['language']]

    # Filter the training data for the current language
    language_df = df[df['language'] == language_data['language']]

    # If fewer examples are available than requested, reduce the number
    num_available_examples = len(language_df)
    num_examples = min(num_examples, num_available_examples)

    # Randomly select `num_examples` examples from the training data
    if num_examples > 0:  # Only sample if there are examples to sample from
      examples_df = language_df.sample(n=num_examples, random_state=42)
      # Build example string for this language
      example_string = ""
      for index, row in examples_df.iterrows():
          example_string += f"Sentence: \"{row['sentence']}\"\nSentiment: {row['label']}\n\n"
    else:
      example_string = "No training examples available for this language.\n\n"


    prompt = f"""Task: Sentiment Analysis in {full_language}

Context: You are an expert in multilingual sentiment analysis, particularly skilled in Indian languages. Your task is to analyze the sentiment of a given sentence in {full_language}.

Here are some examples to guide you:
{example_string}

Input Sentence: "{language_data['sentence']}"

Instructions:
1. Carefully read and understand the sentence in {full_language}.
2. Consider the cultural and linguistic nuances specific to {full_language}.
3. Analyze the overall tone, emotion, and context of the sentence.
4. Classify the sentiment as either 'Positive' or 'Negative'.
5. Provide your answer in the exact format specified below.

Output Format:
The sentiment of the given {full_language} sentence is: <SENTIMENT>

Important Notes:
- Only use 'Positive' or 'Negative' as your sentiment classification.
- Do not provide any explanations or additional commentary.
- Ensure your response adheres strictly to the output format.

Your response:
The sentiment of the given {full_language} sentence is: """

    return prompt

In [10]:
# Define Alpaca-style prompt template with few-shot examples
def create_few_shot_prompt(sentence, label, language, example_df, num_examples=2):
  
  # Filter the example DataFrame for the current language
  language_df = example_df[example_df["language"] == language]

  # If there are fewer examples available than requested, reduce the number
  num_available_examples = len(language_df)
  num_examples = min(num_examples, num_available_examples)

  # Randomly sample `num_examples` from the filtered DataFrame.
  if num_examples > 0: # Only sample if there are examples
    examples = language_df.sample(n=num_examples, random_state=42)  # Consistent sampling
  else:
    examples = pd.DataFrame() # Create an empty dataframe so it doesn't break downstream

  # Build the prompt string, including the few-shot examples
  prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

"""
  # Add the examples in alpaca format.
  for _, row in examples.iterrows():
      example_prompt = f"""### Instruction: Perform sentiment analysis on the following sentence, positive or negative.
### Input: {row['sentence']}
### Response: {row['label']}

"""
      prompt += example_prompt # append example.

  prompt += f"""### Instruction: Perform sentiment analysis on the following sentence, positive or negative.
### Input: {sentence}
### Response: {label}"""
  return prompt

df = pd.read_csv("/kaggle/input/multi-lingual-sentiment-analysis/train.csv")

def formatting_prompts_func(examples, train_df, num_examples = 2):
    """
    Formats examples for Supervised Fine Tuning.

    Args:
        examples (dict): A dictionary of examples, including 'sentence', 'label',
                         and 'language'.
        train_df (pd.DataFrame): The training data DataFrame to draw examples from.
    Returns:
        dict: A dictionary with 'input_ids', 'attention_mask', and 'labels'.
    """
    texts = []
    for sentence, label, language in zip(examples["sentence"], examples["label"], examples["language"]):
        text = create_few_shot_prompt(sentence, label, language, train_df, num_examples) + tokenizer.eos_token # use tokenizer here to make sure it's defined.
        texts.append(text)


    # Tokenize with padding & truncation enabled
    tokenized_inputs = tokenizer(
        texts,
        padding="max_length",  # Ensures batch compatibility
        truncation=True,
        max_length=max_seq_length,
        return_tensors="np",  # Convert to NumPy to avoid tensor issues
    )


    return {
        "input_ids": tokenized_inputs["input_ids"].tolist(),
        "attention_mask": tokenized_inputs["attention_mask"].tolist(),
        "texts": texts
    }

# Apply tokenization with batched processing
dataset = dataset_train.map(lambda example: formatting_prompts_func(example, df , num_examples=2), batched=True, remove_columns=dataset_train.column_names)


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [11]:
dataset[0]['texts']

'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction: Perform sentiment analysis on the following sentence, positive or negative.\n### Input: "এসি চেয়ার কার এবং 3 টিয়ার এসি স্লিপারের থার্মোস্ট্যাটিক কন্ট্রোলগুলি ঘন ঘন পরীক্ষা করা প্রয়োজন কারণ অনেক বার তাপমাত্রা হয় খুব কম বা খুব বেশি..."\n### Response: Negative\n\n### Instruction: Perform sentiment analysis on the following sentence, positive or negative.\n### Input: খুব ছোট এলাকা এবং সঙ্গীত উচ্চমানের নয় এবং খুব জোরে যা একটি ছোট এলাকায় কানের জন্য অস্বস্তিকর। প্রবেশ মূল্য অনেক বেশি (জনপ্রতি 1000 টাকা)\n### Response: Negative\n\n### Instruction: Perform sentiment analysis on the following sentence, positive or negative.\n### Input: কর্মীদের ভাল আচরণ এবং খাবারের পাশাপাশি পানীয় (ককটেল এবং মকটেল) সহ একটি অনন্য জায়গা খুবই ভাল। প্রায়ই একটি সরাসরি সঙ্গীত পরিবেশনের সাথে এমন পরিবেশ তৈরী করে যে একজন দিন এবং সন্ধ্যা উভয় সম

In [12]:
from trl import SFTTrainer

from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "texts",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        
        # Use num_train_epochs = 1, warmup_ratio for full training runs!
        warmup_steps = 5,
        max_steps = 60,

        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

# TrainingArguments(
#         per_device_train_batch_size = 2,
#         gradient_accumulation_steps = 4,
#         warmup_steps = 5,
#         max_steps = 20,
#         learning_rate = 2e-4,
#         fp16 = not torch.cuda.is_bf16_supported(),
#         bf16 = torch.cuda.is_bf16_supported(),
#         logging_steps = 1,
#         optim = "adamw_8bit",
#         weight_decay = 0.01,
#         lr_scheduler_type = "linear",
#         seed = 3407,
#         output_dir = "outputs",
#         report_to = "none", # Use this for WandB etc
#     ),

Applying chat template to train dataset (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [13]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,0.894600
2,0.991800
3,1.045100
4,0.952800
5,1.231400
6,0.818900
7,0.997700
8,1.104300
9,1.118400
10,1.015300


In [14]:
test_df = pd.read_csv('/kaggle/input/multi-lingual-sentiment-analysis/test.csv')
test_df.head()

,ID,sentence,language
0,1,"1120 mAh, ਓਵਰਚਾਰਜਿੰਗ ਦੀ ਸੁਰੱਖਿਆ",pa
1,2,તે સઘન મોઇશ્ચરાઇઝિંગ પ્રદાન કરે છે અને સરસ સ્વ...,gu
2,3,"1120 ಎಂಎಎಚ್, ಮಿತಿಮೀರಿದ ರಕ್ಷಣೆ",kn
3,4,ভাৰতত নিৰ্মিত সৰ্বশ্ৰেষ্ঠ পাৰফিউম ব্ৰেণ্ডবোৰৰ ...,as
4,5,"میں نے حال ہی میں ""انفولڈ"" سے ایک ٹیمپلیٹ خرید...",ur


In [15]:
FastLanguageModel.for_inference(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

In [16]:
def predict_sentiment(row, model, tokenizer, train_df, num_examples=2, max_new_tokens=50):
    
    sentence = row['sentence']
    language = row['language']

    # Create the few-shot prompt
    prompt = create_few_shot_prompt(sentence, language, train_df, num_examples)

    # Encode the prompt
    inputs = tokenizer(
        [prompt],
        return_tensors="pt",
        truncation=True,
        padding=True
    ).to("cuda")

    # Generate prediction
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_new_tokens=max_new_tokens,  # Limit the length of the generated sentiment
            use_cache=True,
        )

    # Decode and extract the response
    generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    prediction = generated_text.split("### Response:")[-1].strip()

    # Clean and standardize the prediction (important!)
    prediction = prediction.lower()
    if "positive" in prediction:
        return "Positive"
    # elif "negative" in prediction:
    #     return "Negative"
    else:
        return "Negative"

def create_few_shot_prompt(sentence, language, example_df, num_examples=2):
    

    # Filter the example DataFrame for the current language
    language_df = example_df[example_df["language"] == language]

    # If there are fewer examples available than requested, reduce the number
    num_available_examples = len(language_df)
    num_examples = min(num_examples, num_available_examples)

    # Randomly sample `num_examples` from the filtered DataFrame.
    if num_examples > 0: # Only sample if there are examples
      examples = language_df.sample(n=num_examples, random_state=42)  # Consistent sampling
    else:
      examples = pd.DataFrame() # Create an empty dataframe so it doesn't break downstream

    # Build the prompt string, including the few-shot examples
    prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

"""
    # Add the examples in alpaca format.
    for _, row in examples.iterrows():
        example_prompt = f"""### Instruction: Perform sentiment analysis on the following sentence, positive or negative.
### Input: {row['sentence']}
### Response: {row['label']}

"""
        prompt += example_prompt # append example.

    prompt += f"""### Instruction: Perform sentiment analysis on the following sentence, positive or negative.
### Input: {sentence}
### Response: """  # Leave the response blank - this is inference.
    return prompt

map_dict = {
    "as": "Assamese", "bd": "Bodo", "bn": "Bengali", "gu": "Gujarati",
    "hi": "Hindi", "kn": "Kannada", "ml": "Malayalam", "mr": "Marathi",
    "or": "Odia", "pa": "Punjabi", "ta": "Tamil", "te": "Telugu", "ur": "Urdu"
}



# set tokenizer pad token
tokenizer.pad_token = tokenizer.eos_token

test_df['label'] = test_df.apply(lambda row: predict_sentiment(row, model, tokenizer, df, num_examples=2), axis=1)  # Pass train_df


In [17]:
# Prepare submission file
submission = test_df[['ID', 'label']]
submission.to_csv('submission.csv', index=False)

print(f"Submission file created: submission.csv")

Submission file created: submission.csv


In [18]:
submission_df = pd.read_csv("/kaggle/working/submission.csv")
submission_df

,ID,label
0,1,Negative
1,2,Positive
2,3,Negative
3,4,Positive
4,5,Negative
...,...,...
95,96,Positive
96,97,Positive
97,98,Negative
98,99,Positive


In [19]:
submission_df['label'].value_counts()

label
Negative    56
Positive    44
Name: count, dtype: int64